In [2]:
import json
import requests
import numpy as np
import pandas as pd
import requests

In [3]:
credentials = json.loads(open('credentials.json').read())
token =  credentials['token']

In [4]:
headers = {'Authorization': 'token ' + token}
data = requests.get('https://api.github.com/user', headers=headers)
data = data.json()

In [5]:
url = data['repos_url']

In [6]:
url = data['repos_url']
page_no = 1
repos_data = []
while (True):
    response = requests.get(url, headers=headers)
    response = response.json()
    repos_data = repos_data + response
    repos_fetched = len(response)
    print("Total repositories fetched: {}".format(repos_fetched))
    if (repos_fetched == 30):
        page_no = page_no + 1
        url = data['repos_url'].encode("UTF-8") + '?page=' + str(page_no)
    else:
        break

Total repositories fetched: 15


In [7]:
#repos_data[5]

In [8]:
repos_information = []
for repo in repos_data:
    data = []
    data.append(repo['id'])
    data.append(repo['name'])
    data.append(repo['description'])
    data.append(repo['created_at'])
    data.append(repo['updated_at'])
    data.append(repo['owner']['login'])
    data.append(repo['watchers_count'])
    data.append(repo['url'])
    data.append(repo['commits_url'].split("{")[0])
    data.append(repo['languages_url'])
    repos_information.append(data)

In [9]:

repo_df_column = ['Id','Name','Description','Created on','Updated on','Owner','Watchers count', 
                    'Url','Commits url','Languages url']

In [14]:
repos_df = pd.DataFrame(repos_information,columns=repo_df_column)
type(repos_df)

pandas.core.frame.DataFrame

In [11]:
for i in range(len(repos_df)):
    language_url = repos_df.loc[i,'Languages url']
    language_response = requests.get(language_url, headers=headers)
    language_response =  language_response.json()
    if language_response != {}:
        language=[]
        for key,value in language_response.items():
            language.append(key)
        language = ', '.join(language)
        repos_df.loc[i,'Language'] = language
    else:
        repos_df.loc[i,'Language'] = ""

In [12]:
repos_df.to_csv('repos_info.csv', index = False)

In [13]:
commit_information= []
for index, row in repos_df.iterrows():
    commit_url = row['Commits url']
    commit_response = requests.get(commit_url,headers=headers)
    commit_response = commit_response.json()
    for commit in commit_response:
        commit_data = []
        commit_data.append(repos_df.loc[index,"Id"])
        commit_data.append(commit['sha'])
        commit_data.append(commit['commit']['message'])
        commit_data.append(commit['commit']['author']['date'])
        commit_information.append(commit_data)


In [33]:
commit_information

[[234071231,
  '834843d5307f829103a185e29ea9df0de279bda2',
  'Updated the Index.html',
  '2020-01-15T12:13:41Z'],
 [234071231,
  'b69b1517f299958fc24a3a897890197d7eb74dd5',
  'READMe.md file commited',
  '2020-01-15T12:04:52Z'],
 [234071231,
  '9072be80ea4de086796e72291627b21024e21e13',
  'Initial Commit',
  '2020-01-15T12:03:13Z'],
 [239988172,
  '89fe55999449365d4d026c00ce7bfcf587ed59bc',
  'updated the index.html',
  '2020-02-12T11:20:24Z'],
 [239988172,
  '3411437305d486593b14a01693fe92bec2b61208',
  "Merge branch 'master' of https://github.com/animeshsingh04/AngularProject-to-showApilog-from-file",
  '2020-02-12T10:52:02Z'],
 [239988172,
  '65c5c004c5cad46b84aeb8207be897acdb421161',
  'commit message',
  '2020-02-12T10:47:02Z'],
 [239988172,
  '3e74f94372caac21168be3cf6635a3920e6136c1',
  'Initial commit',
  '2020-02-12T10:43:18Z'],
 [233563609,
  'a1b2f405fe26884d0179ab0bb893de1a794284c2',
  'Update README.md',
  '2020-01-13T12:45:53Z'],
 [233563609,
  '7c1aca56bd7d573fc1ffd56e40

In [34]:
commit_df_column = ['Id','Sha','Commit Message','Commited on']

In [35]:
commit_df = pd.DataFrame(commit_information,columns=commit_df_column)

In [36]:
commit_df.to_csv('commit_info.csv',index = False)